<hr style="border:2px solid #0281c9"> </hr>

<img align="left" alt="ESO Logo" src="http://archive.eso.org/i/esologo.png">  

<div align="center">
  <h1 style="color: #0281c9; font-weight: bold;">Astroquery Beta Test</h1>
  <h2 style="color: #0281c9; font-weight: bold;">ESO Science Archive - April 2025</h2> 
</div>

<hr style="border:2px solid #0281c9"> </hr>

_This notebook focuses specifically on testing recent developments related to the transition from the WDB backend to the TAP backend._ - [DFS-19635](https://jira.eso.org/browse/DFS-19635)

As a Beta tester, we would like to ask your feedback on the following aspects:

 - What absolutely needs to be fixed before the release
 - What would be nice to be fixed before the release
 - Feature ideas and improvements for subsequent releases

The current astroquery ESO module documentation will mostly work: https://astroquery.readthedocs.io/en/latest/eso/eso.html

For the parts that unevitably needed changes, the updated documentation is found in the `docs` folder in this repository:
 - `./docs/AstroqueryEsoDocumentation-Beta.pdf`
 - `./docs/AstroqueryEsoDocstrings-Beta.pdf`
 - `./docs/html/`


## Installation

In [ ]:
!bash install_astroquery.sh

## Handy imports and functions

In [ ]:
import os
import pandas as pd
import astropy.table.table as aqtable
from pyvo.dal.exceptions import DALQueryError, DALServiceError
from IPython.display import clear_output

def table_to_csv(aqtable: aqtable, filepath: str):
    aqtable.to_pandas().to_csv(filepath)
    print(f"Table saved to {filepath}")

## ESO instance

In [ ]:
from astroquery.eso import Eso

eso_instance = Eso()

eso_instance.maxrec = 1000

print(f"ESO tap_url = {eso_instance._tap_url()}")

# Tutorial

## Query Raw Data

In [ ]:
my_table = eso_instance.query_main()
table_to_csv(my_table, "./eso_aq_example_query_main.csv")
# my_table[:2]  # Uncomment to show the table here in the notebook

## Query Phase 3 Data

In [ ]:
available_surveys = eso_instance.list_surveys()

print(pd.DataFrame({"survey": available_surveys}).to_string())

In [ ]:
my_table = eso_instance.query_surveys()

table_to_csv(my_table, "./eso_aq_example_query_surveys.csv")
# my_table[:2]  # Uncomment to show the table here in the notebook

In [ ]:
my_table = eso_instance.query_surveys('SPHERE')

table_to_csv(my_table, "./eso_aq_example_query_surveys_sphere.csv")
# my_table  # Uncomment to show the table here in the notebook

In [ ]:
my_table = eso_instance.query_surveys(['SPHERE', 'VEGAS'])
my_table_2 = eso_instance.query_surveys('SPHERE, VEGAS')
assert all(my_table.values_equal(my_table_2))

table_to_csv(my_table, "./eso_aq_example_query_surveys_sphere_vegas.csv")

#my_table  # Uncomment to show the table here in the notebook

## Query Instrument Specific Data

In [ ]:
available_instruments = eso_instance.list_instruments()
print(pd.DataFrame({"instrument": available_instruments}).to_string())

In [ ]:
my_table = eso_instance.query_instrument("midi")
table_to_csv(my_table, "./eso_aq_example_query_instrument_midi.csv")
# my_table  # Uncomment to show the table here in the notebook

## Query Apex Quicklooks

In [ ]:
tblb = eso_instance.query_apex_quicklooks(project_id='E-095.F-9802A-2015')
tbla = eso_instance.query_apex_quicklooks(prog_id='095.F-9802(A)')

assert len(tbla) == 5
assert set(tbla['release_date']) == {
    '2015-07-17T03:06:23.280Z',
    '2015-07-18T12:07:32.713Z',
    '2015-09-18T11:31:15.867Z',
    '2015-09-15T11:06:55.663Z',
    '2015-09-18T11:46:19.970Z'
}

## Info before launching a query

### Rows and columns

In [ ]:
eso_instance.query_main(help=True)

### Number of records

In [ ]:
count_m = eso_instance.query_main(count_only=True)
count_n = eso_instance.query_main(instrument="MUSE", count_only=True)
print(f"Counts: {count_m:0.2e}, {count_n:0.2e}")

count_m = eso_instance.query_surveys('MUSE', count_only=True)
count_n = eso_instance.query_surveys('MUSE', cone_ra=18, cone_dec=0 , cone_radius=1, count_only=True)
print(f"Counts: {count_m:0.2e}, {count_n:0.2e}")

In [ ]:
survey_nrows = {"survey": [], "num_records": []}
for x in available_surveys:
    n = eso_instance.query_surveys(x, count_only=True)
    survey_nrows["num_records"].append(n)
    survey_nrows["survey"].append(x)
    print(x, n)

clear_output()
print(pd.DataFrame(survey_nrows).to_string())

### Column names as list

In [ ]:
column_list = list(eso_instance.query_main(top=1).columns)
print(pd.DataFrame(column_list).to_string())

## Limit the columns to query

In [ ]:
my_table = eso_instance.query_main(columns=["target", "instrument", "date_obs", "access_url", "datalink_url"])
table_to_csv(my_table, "./eso_aq_example_query_main_columns.csv")
#my_table[0:3]  # Uncomment to show the table here in the notebook

## Limit the number of records - _top_ vs _maxrec_

 - `top`: "I don't care how many records there are, give me only the _top N_, because my science requires only those."
 - `maxrec`: "Give me as many records as possible given my computational resources and time constraints. Please warn me if results are truncated."

Key Differences:

 - Scope: `TOP` is used within the ADQL query to limit the number of rows returned by that specific query. In contrast, `maxrec` is a TAP service parameter that sets an upper limit on the number of records any query can return, acting as a server-side `safeguard` control.

 - Implementation: `TOP` is specified in the query language (ADQL), whereas `maxrec` is a parameter of the TAP service protocol, often set outside the query itself.

In [ ]:
eso_instance.maxrec = None 
eso_instance.query_main(top=3)

In [ ]:
eso_instance.maxrec = 3
eso_instance.query_main()

In [ ]:
eso_instance.maxrec = 5
eso_instance.query_main(top=10)

In [ ]:
eso_instance.maxrec = 10
eso_instance.query_main(top=5)

In [ ]:
eso_instance.maxrec = 1000

## Filter in advance

In [ ]:
my_table = eso_instance.query_surveys("sphere, vegas",
                                          columns="obs_collection, calib_level, multi_ob, filter, s_pixel_scale, instrument_name" ,
                                          calib_level=3,
                                          multi_ob='M')

table_to_csv(my_table, "./eso_aq_example_filter_vegas.csv")

## Filter afterwards

In [ ]:
my_table[my_table["s_pixel_scale"] > 0.2]

## Free ADQL query

In [ ]:
eso_instance.query_tap_service("Select obs_collection, count(*) from ivoa.ObsCore group by obs_collection order by obs_collection")

In [ ]:
eso_instance.query_tap_service("Select obs_collection, calib_level, multi_ob, filter, s_pixel_scale, instrument_name "
                               "from ivoa.ObsCore where "
                               "obs_collection in ('sphere', 'vegas') and "
                               "calib_level=3 and "
                               "multi_ob='M' and "
                               "s_pixel_scale < 0.2")

## Authenticated queries

 - Even when the login is correct, the query is unauthenticated.
 - No authenticated queries possible --> No risk of data leaks.

In [ ]:
eso_instance.login(username='your_username')
query_str = "select count(*) from dbo.raw"

In [ ]:
count_unauthenticated = eso_instance.query_tap_service(query_str)
print(count_unauthenticated) # 3453xxxx

In [ ]:
# Takes longer - 17s instead of 0.5s
count_authenticated = eso_instance.query_tap_service(query_str, authenticated = True)
print(count_authenticated) # 4839xxxx

## Print the ADQL string without issuing the query

In [ ]:
q_str = eso_instance.query_main(asdasd='asdasd', query_str_only=True)
print(q_str)

q_str = eso_instance.query_main(cone_ra=18, cone_dec=0 , cone_radius=0.1, query_str_only=True)
print(q_str)

q_str = eso_instance.query_main(cone_ra=18, cone_dec=0 , cone_radius=0.1, asdasd='asdasd', query_str_only=True)
print(q_str)

## Cone Search

In [ ]:
my_table = eso_instance.query_surveys(surveys='MUSE', cone_ra=150, cone_dec=-59, cone_radius=1, columns="target_name, s_ra, s_dec, s_region")
table_to_csv(my_table, "./eso_aq_example_cone_search_phase_3_muse.csv")

In [ ]:
my_table = eso_instance.query_surveys(cone_ra=150, cone_dec=-59, cone_radius=1, columns="target_name, s_ra, s_dec, s_region")
table_to_csv(my_table, "./eso_aq_example_cone_search_phase_3_all.csv")

In [ ]:
# - Qery survey - # - Phase 3 data

count_m = eso_instance.query_surveys('MUSE', cone_ra=18, cone_dec=0 , cone_radius=1, count_only=True, query_str_only=False)
print("Phase 3 MUSE: ", count_m)
print()

count_m = eso_instance.query_surveys(cone_ra=18, cone_dec=0 , cone_radius=1, count_only=True, query_str_only=False)
print("Phase 3 ALL: ", count_m)
print()

# - Query main - # - Raw Data

count_m = eso_instance.query_main('MUSE', cone_ra=18, cone_dec=0 , cone_radius=1, count_only=True, query_str_only=False)
print("Raw MUSE: ", count_m)
print()

count_m = eso_instance.query_main(cone_ra=18, cone_dec=0 , cone_radius=1, count_only=True, query_str_only=False)
print("Raw ALL: ", count_m)
print()

# - Query instrument - # - Instrument Specific Raw Data

count_m = eso_instance.query_instrument('MUSE', cone_ra=18, cone_dec=0 , cone_radius=1, count_only=True, query_str_only=False)
print("Instrument Specific Raw - MUSE: ", count_m)
print()

## Beta Tester Feedback:

### General free-hand comments and impressions:

...

### Did something break?

 - query_instrument('MUSA') - It broke when I tried to query for the instrument 'MUSA'
 - ...

### Could something be better?

 - query_main() - I don't like the name of this function because...
 - ...


## Features that I'd like the module to support:

Add the features you're like to be implemented. If the feature you want is already listed, please put a star (*) next to it.
 
 - query_catalogues
 - SkyCoord
 - query_region
 - query_object
 - OR, >, <, between, like, ...
 - Uncorrelated random sample queries
 - cutouts - definition and download
 - asyncronous queries
 - ? previews, ssap, datalink
 - parameter `pint_query = True` prints the query executed. Example: `query_main(..., print_query=True)`
 - sort results (order by)

<hr style="border:2px solid #0281c9"> </hr>

<div align="center">
  <h1 style="color: #0281c9; font-weight: bold;">Run your own experiments</h1>
</div>

<hr style="border:2px solid #0281c9"> </hr>

In [ ]:
# Experiment with your own queries